In [1]:
import requests
from bs4 import BeautifulSoup
from google.colab import drive
import os
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import fitz
import faiss
import numpy as np
import json
from tqdm import tqdm, trange
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

***PRE-PROCESSING***

Disease

**Common Diseases**
Diabetes Mellitus
Hypertension (High Blood Pressure)
Asthma
Chronic Obstructive Pulmonary Disease (COPD)
Heart Disease (Coronary Artery Disease)
Stroke
Cancer (various types)
HIV/AIDS
Alzheimer's Disease
Arthritis (Rheumatoid and Osteoarthritis)
**Infectious Diseases**
Influenza
Tuberculosis
COVID-19
Hepatitis (A, B, C)
Malaria
Measles
Chickenpox
Pneumonia
Ebola Virus Disease
Zika Virus
**Autoimmune Diseases**
Multiple Sclerosis
Systemic Lupus Erythematosus (Lupus)
Type 1 Diabetes
Hashimoto's Thyroiditis
Celiac Disease
Graves' Disease
Psoriasis
Crohn's Disease
Ulcerative Colitis
Rheumatoid Arthritis
**Mental Health Disorders**
Depression
Anxiety Disorders (Generalized Anxiety Disorder, Panic Disorder)
Bipolar Disorder
Schizophrenia
Obsessive-Compulsive Disorder (OCD)
Post-Traumatic Stress Disorder (PTSD)
Attention-Deficit/Hyperactivity Disorder (ADHD)
Autism Spectrum Disorders
Eating Disorders (Anorexia, Bulimia)
Personality Disorders
**Rare Diseases**
Cystic Fibrosis
Sickle Cell Disease
Huntington's Disease
Amyotrophic Lateral Sclerosis (ALS)
Tourette Syndrome
Fibrodysplasia Ossificans Progressiva
Pompe Disease
Hemophilia
Marfan Syndrome
Wilson's Disease
**Chronic Diseases**
Chronic Kidney Disease
Chronic Liver Disease (Cirrhosis)
Osteoporosis
Gout
Fibromyalgia
Chronic Fatigue Syndrome
Irritable Bowel Syndrome (IBS)
Sleep Apnea
Chronic Migraines
Non-Alcoholic Fatty Liver Disease (NAFLD)

In [4]:
#Urls for above mentioned diseases
disease_urls = [
    'https://www.mayoclinic.org/diseases-conditions/diabetes/symptoms-causes/syc-20371444#:~:text=Diabetes%20mellitus%20refers%20to%20a,of%20diabetes%20varies%20by%20type.',
    'https://www.mayoclinic.org/diseases-conditions/high-blood-pressure/symptoms-causes/syc-20373410',
    'https://www.mayoclinic.org/diseases-conditions/asthma/symptoms-causes/syc-20369653',
    'https://www.mayoclinic.org/diseases-conditions/copd/symptoms-causes/syc-20353679',
    'https://www.mayoclinic.org/diseases-conditions/coronary-artery-disease/symptoms-causes/syc-20350613#:~:text=CAD%20happens%20when%20coronary%20arteries,or%20even%20a%20heart%20attack.',
    'https://www.mayoclinic.org/diseases-conditions/stroke/symptoms-causes/syc-20350113',
    'https://stanfordhealthcare.org/medical-conditions/cancer/cancer/cancer-types.html',
    'https://www.mayoclinic.org/diseases-conditions/hiv-aids/symptoms-causes/syc-20373524',
    'https://www.mayoclinic.org/diseases-conditions/alzheimers-disease/symptoms-causes/syc-20350447',
    'https://www.jnj.com/health-and-wellness/arthritis-3-main-types-rheumatoid-psoriatic-osteoarthritis',
    'https://www.cdc.gov/flu/index.html',
    'https://www.mayoclinic.org/diseases-conditions/tuberculosis/symptoms-causes/syc-20351250',
    'https://en.wikipedia.org/wiki/COVID-19',
    'https://www.cdc.gov/hepatitis/about/index.html',
    'https://www.who.int/news-room/fact-sheets/detail/malaria#:~:text=Malaria%20is%20a%20life%2Dthreatening,spread%20from%20person%20to%20person.',
    'https://www.who.int/news-room/fact-sheets/detail/measles#:~:text=Measles%20infects%20the%20respiratory%20tract,body%20fight%20off%20the%20virus.',
    'https://www.cdc.gov/chickenpox/about/index.html#:~:text=Chickenpox%20is%20a%20highly%20contagious%20disease%20caused%20by%20varicella%2Dzoster,chickenpox%20will%20also%20get%20infected.',
    'https://www.mayoclinic.org/diseases-conditions/pneumonia/symptoms-causes/syc-20354204#:~:text=Pneumonia%20is%20an%20infection%20that,and%20fungi%2C%20can%20cause%20pneumonia.',
    'https://www.who.int/news-room/fact-sheets/detail/ebola-virus-disease',
    'https://www.who.int/news-room/fact-sheets/detail/zika-virus',
    'https://www.hopkinsmedicine.org/health/conditions-and-diseases/multiple-sclerosis-ms#:~:text=Multiple%20sclerosis%20(MS)%20is%20a%20chronic%20disease%20of%20the%20central,trouble%20walking%2C%20and%20tingling%20feelings.',
    'https://www.niams.nih.gov/health-topics/lupus#:~:text=Systemic%20lupus%20erythematosus%20(lupus)%20is,disease%2C%20attacks%20its%20own%20tissues.',
    'https://www.mayoclinic.org/diseases-conditions/type-1-diabetes/symptoms-causes/syc-20353011',
    'https://www.mayoclinic.org/diseases-conditions/hashimotos-disease/symptoms-causes/syc-20351855',
    'https://www.mayoclinic.org/diseases-conditions/celiac-disease/symptoms-causes/syc-20352220',
    'https://www.mayoclinic.org/diseases-conditions/graves-disease/symptoms-causes/syc-20356240',
    'https://www.psoriasis.org/about-psoriasis/',
    'https://en.wikipedia.org/wiki/Crohn%27s_disease',
    'https://en.wikipedia.org/wiki/Ulcerative_colitis',
    'https://en.wikipedia.org/wiki/Rheumatoid_arthritis',
    'https://en.wikipedia.org/wiki/Depression_(mood)',
    'https://en.wikipedia.org/wiki/Generalized_anxiety_disorder','https://en.wikipedia.org/wiki/Panic_disorder',
    'https://en.wikipedia.org/wiki/Bipolar_disorder','https://en.wikipedia.org/wiki/Schizophrenia',
    'https://en.wikipedia.org/wiki/Obsessive%E2%80%93compulsive_disorder','https://www.mayoclinic.org/diseases-conditions/post-traumatic-stress-disorder/symptoms-causes/syc-20355967',
    'https://en.wikipedia.org/wiki/Attention_deficit_hyperactivity_disorder',
    'https://en.wikipedia.org/wiki/Autism','https://en.wikipedia.org/wiki/Anorexia_nervosa','https://en.wikipedia.org/wiki/Bulimia_nervosa',
    'https://en.wikipedia.org/wiki/Personality_disorder','https://www.mayoclinic.org/diseases-conditions/cystic-fibrosis/symptoms-causes/syc-20353700',
    'https://www.mayoclinic.org/diseases-conditions/sickle-cell-anemia/symptoms-causes/syc-20355876','https://www.mayoclinic.org/diseases-conditions/huntingtons-disease/symptoms-causes/syc-20356117',
    'https://www.mayoclinic.org/diseases-conditions/amyotrophic-lateral-sclerosis/symptoms-causes/syc-20354022',
    'https://www.mayoclinic.org/diseases-conditions/tourette-syndrome/symptoms-causes/syc-20350465',
    'https://en.wikipedia.org/wiki/Fibrodysplasia_ossificans_progressiva','https://my.clevelandclinic.org/health/diseases/15808-pompe-disease',
    'https://www.mayoclinic.org/diseases-conditions/hemophilia/symptoms-causes/syc-20373327',
    'https://www.mayoclinic.org/diseases-conditions/marfan-syndrome/symptoms-causes/syc-20350782#:~:text=Marfan%20syndrome%20most%20commonly%20affects,can%20be%20mild%20or%20severe.',
    'https://my.clevelandclinic.org/health/diseases/5957-wilson-disease',
    'https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521',
    'https://my.clevelandclinic.org/health/diseases/15572-cirrhosis-of-the-liver',
    'https://www.niams.nih.gov/health-topics/osteoporosis','https://my.clevelandclinic.org/health/diseases/4755-gout',
    'https://www.niams.nih.gov/health-topics/fibromyalgia','https://www.mayoclinic.org/diseases-conditions/chronic-fatigue-syndrome/symptoms-causes/syc-20360490#:~:text=Myalgic%20encephalomyelitis%2Fchronic%20fatigue%20syndrome%20(ME%2FCFS)%20is,although%20there%20are%20many%20theories.',
    'https://my.clevelandclinic.org/health/diseases/4342-irritable-bowel-syndrome-ibs',
    'https://www.mayoclinic.org/diseases-conditions/sleep-apnea/symptoms-causes/syc-20377631',
    'https://www.mayoclinic.org/diseases-conditions/migraine-headache/symptoms-causes/syc-20360201',
    'https://www.nhs.uk/conditions/non-alcoholic-fatty-liver-disease/#:~:text=Non%2Dalcoholic%20fatty%20liver%20disease%20(NAFLD)%20is%20the%20term,cirrhosis%2C%20if%20it%20gets%20worse.']



In [5]:
#Webscrapping the websites
output_directory = '/content/drive/MyDrive/Medical_Data/Disease'
os.makedirs(output_directory, exist_ok=True)

def save_text_from_url(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text(separator='\n')
        with open(os.path.join(output_directory, filename), 'w', encoding='utf-8') as f:
            f.write(text)

        print(f"Saved: {filename}")

    except Exception as e:
        print(f"Error saving {filename}: {e}")

for i, url in enumerate(disease_urls):
    save_text_from_url(url, f'disease_info_{i}.txt')


Saved: disease_info_0.txt
Saved: disease_info_1.txt
Saved: disease_info_2.txt
Saved: disease_info_3.txt
Saved: disease_info_4.txt
Saved: disease_info_5.txt
Saved: disease_info_6.txt
Saved: disease_info_7.txt
Saved: disease_info_8.txt
Saved: disease_info_9.txt
Saved: disease_info_10.txt
Saved: disease_info_11.txt
Saved: disease_info_12.txt
Saved: disease_info_13.txt
Saved: disease_info_14.txt
Saved: disease_info_15.txt
Saved: disease_info_16.txt
Saved: disease_info_17.txt
Saved: disease_info_18.txt
Saved: disease_info_19.txt
Saved: disease_info_20.txt
Saved: disease_info_21.txt
Saved: disease_info_22.txt
Saved: disease_info_23.txt
Saved: disease_info_24.txt
Saved: disease_info_25.txt
Saved: disease_info_26.txt
Saved: disease_info_27.txt
Saved: disease_info_28.txt
Saved: disease_info_29.txt
Saved: disease_info_30.txt
Saved: disease_info_31.txt
Saved: disease_info_32.txt
Saved: disease_info_33.txt
Saved: disease_info_34.txt
Saved: disease_info_35.txt
Saved: disease_info_36.txt
Saved: dise

In [6]:
#Combine the text files to one
def combine_txt_files(input_directory):
    output_directory = '/content/drive/MyDrive/Medical_Data/Disease'
    output_file = os.path.join(output_directory, 'Combined_Disease.txt')

    combined_content = []

    for filename in os.listdir(input_directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(input_directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                combined_content.append(file.read())

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write('\n\n'.join(combined_content))  #

    print(f"Combined file saved to: {output_file}")

input_directory = '/content/drive/MyDrive/Medical_Data/Disease'
combine_txt_files(input_directory)


Combined file saved to: /content/drive/MyDrive/Medical_Data/Disease/Combined_Disease.txt


In [7]:
#Data Pre-Processing
def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Initialize stemmer
    stemmer = PorterStemmer()

    # Stem the tokens
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)

    return preprocessed_text

file_path = '/content/drive/MyDrive/Medical_Data/Disease/Combined_Disease.txt'
preprocessed_data = preprocess_text(file_path)

# Save the preprocessed data to a new file
preprocessed_file_path = '/content/drive/MyDrive/Medical_Data/Disease/Preprocessed_Disease.txt'
with open(preprocessed_file_path, 'w', encoding='utf-8') as file:
    file.write(preprocessed_data)

print(f"Preprocessed data saved to: {preprocessed_file_path}")

Preprocessed data saved to: /content/drive/MyDrive/Medical_Data/Disease/Preprocessed_Disease.txt


Drug Information

**Analgesics**
Acetaminophen (Tylenol)
Ibuprofen (Advil, Motrin)
Aspirin
Naproxen (Aleve)
**Antibiotics**
Amoxicillin
Ciprofloxacin (Cipro)
Azithromycin (Zithromax)
Cephalexin (Keflex)
**Antidepressants**
Fluoxetine (Prozac)
Sertraline (Zoloft)
Escitalopram (Lexapro)
Venlafaxine (Effexor)
**Antihypertensives**
Lisinopril (Prinivil, Zestril)
Amlodipine (Norvasc)
Losartan (Cozaar)
Metoprolol (Lopressor, Toprol-XL)
**Antidiabetics**
Metformin (Glucophage)
Glyburide (Diabeta)
Insulin (various forms)
Sitagliptin (Januvia)
**Antihistamines**
Diphenhydramine (Benadryl)
Loratadine (Claritin)
Cetirizine (Zyrtec)
Fexofenadine (Allegra)
**Antifungals**
Fluconazole (Diflucan)
Clotrimazole (Lotrimin)
Terbinafine (Lamisil)
**Antivirals**
Oseltamivir (Tamiflu)
Acyclovir (Zovirax)
Sofosbuvir (Sovaldi)
**Other Common Medications**
Omeprazole (Prilosec)
Simvastatin (Zocor)
Warfarin (Coumadin)
Albuterol (Ventolin, ProAir)

In [8]:
 drug_urls = [
    'https://medlineplus.gov/druginfo/meds/a681004.html',
    'https://medlineplus.gov/druginfo/meds/a682159.html',
    'https://medlineplus.gov/druginfo/meds/a682878.html#:~:text=Aspirin%20is%20in%20a%20group,and%20cough%20and%20cold%20medications.',
    'https://medlineplus.gov/druginfo/meds/a681029.html#:~:text=Nonprescription%20naproxen%20is%20used%20to,class%20of%20medications%20called%20NSAIDs.',
    'https://medlineplus.gov/druginfo/meds/a685001.html',
    'https://medlineplus.gov/druginfo/meds/a688016.html',
    'https://medlineplus.gov/druginfo/meds/a697037.html',
    'https://my.clevelandclinic.org/health/drugs/20055-cephalexin-capsules-or-tablets',
    'https://medlineplus.gov/druginfo/meds/a689006.html',
    'https://my.clevelandclinic.org/health/drugs/20089-sertraline-tablets',
    'https://my.clevelandclinic.org/health/drugs/18917-escitalopram-tablets',
    'https://my.clevelandclinic.org/health/drugs/20828-venlafaxine-tablets',
    'https://medlineplus.gov/druginfo/meds/a692051.html',
    'https://medlineplus.gov/druginfo/meds/a692044.html',
    'https://medlineplus.gov/druginfo/meds/a695008.html',
    'https://medlineplus.gov/druginfo/meds/a682864.html',
    'https://my.clevelandclinic.org/health/drugs/20966-metformin-tablets',
    'https://medlineplus.gov/druginfo/meds/a684058.html','https://my.clevelandclinic.org/health/body/22601-insulin',
    'https://medlineplus.gov/druginfo/meds/a606023.html',
    'https://medlineplus.gov/druginfo/meds/a682539.html',
    'https://www.rxlist.com/claritin-drug.htm',
    'https://my.clevelandclinic.org/health/drugs/19298-cetirizine-tablets',
    'https://my.clevelandclinic.org/health/drugs/20104-fexofenadine-capsules-or-tablets',
    'https://my.clevelandclinic.org/health/drugs/19527-fluconazole-tablets',
    'https://www.mayoclinic.org/drugs-supplements/clotrimazole-topical-route/description/drg-20063212',
    'https://my.clevelandclinic.org/health/drugs/19379-terbinafine-tablets',
    'https://my.clevelandclinic.org/health/drugs/18630-oseltamivir-capsules',
    'https://my.clevelandclinic.org/health/drugs/20037-acyclovir-capsules-or-tablets',
    'https://medlineplus.gov/druginfo/meds/a614014.html',
    'https://my.clevelandclinic.org/health/drugs/19706-simvastatin-tablets',
    'https://my.clevelandclinic.org/health/treatments/16182-warfarin-a-blood-thinning-drug-what-you-need-to-know-',
    'https://medlineplus.gov/druginfo/meds/a682145.html'

]


In [9]:
#Webscrapping the websites
drug_directory = '/content/drive/MyDrive/Medical_Data/Drug'
os.makedirs(drug_directory, exist_ok=True)

def save_text_from_url(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text(separator='\n')
        with open(os.path.join(drug_directory, filename), 'w', encoding='utf-8') as f:
            f.write(text)

        print(f"Saved: {filename}")

    except Exception as e:
        print(f"Error saving {filename}: {e}")

for i, url in enumerate(drug_urls):
    save_text_from_url(url, f'drug_info_{i}.txt')


Saved: drug_info_0.txt
Saved: drug_info_1.txt
Saved: drug_info_2.txt
Saved: drug_info_3.txt
Saved: drug_info_4.txt
Saved: drug_info_5.txt
Saved: drug_info_6.txt
Saved: drug_info_7.txt
Saved: drug_info_8.txt
Saved: drug_info_9.txt
Saved: drug_info_10.txt
Saved: drug_info_11.txt
Saved: drug_info_12.txt
Saved: drug_info_13.txt
Saved: drug_info_14.txt
Saved: drug_info_15.txt
Saved: drug_info_16.txt
Saved: drug_info_17.txt
Saved: drug_info_18.txt
Saved: drug_info_19.txt
Saved: drug_info_20.txt
Saved: drug_info_21.txt
Saved: drug_info_22.txt
Saved: drug_info_23.txt
Saved: drug_info_24.txt
Saved: drug_info_25.txt
Saved: drug_info_26.txt
Saved: drug_info_27.txt
Saved: drug_info_28.txt
Saved: drug_info_29.txt
Saved: drug_info_30.txt
Saved: drug_info_31.txt
Saved: drug_info_32.txt


In [10]:
#Combine the text files to one
def combine_txt_files(input_directory):
    output_directory = '/content/drive/MyDrive/Medical_Data/Drug'
    output_file = os.path.join(output_directory, 'Combined_Drug.txt')

    combined_content = []

    for filename in os.listdir(input_directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(input_directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                combined_content.append(file.read())

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write('\n\n'.join(combined_content))  #

    print(f"Combined file saved to: {output_file}")

input_directory = '/content/drive/MyDrive/Medical_Data/Drug'
combine_txt_files(input_directory)


Combined file saved to: /content/drive/MyDrive/Medical_Data/Drug/Combined_Drug.txt


In [11]:
#Data Pre-Processing
def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Initialize stemmer
    stemmer = PorterStemmer()

    # Stem the tokens
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)

    return preprocessed_text

file_path = '/content/drive/MyDrive/Medical_Data/Drug/Combined_Drug.txt'
preprocessed_data = preprocess_text(file_path)

# Save the preprocessed data to a new file
preprocessed_file_path = '/content/drive/MyDrive/Medical_Data/Drug/Preprocessed_Drug.txt'
with open(preprocessed_file_path, 'w', encoding='utf-8') as file:
    file.write(preprocessed_data)

print(f"Preprocessed data saved to: {preprocessed_file_path}")

Preprocessed data saved to: /content/drive/MyDrive/Medical_Data/Drug/Preprocessed_Drug.txt


Research Papers

* The Role of Clinical Decision Support Systems in Enhancing Patient Care
* Efficacy of Telehealth in Rural Healthcare Delivery
* Advancements in Precision Medicine: Tailoring Treatments to Genetic Profiles
* The Impact of Artificial Intelligence on Radiology and Imaging
* Challenges and Innovations in Electronic Health Record (EHR) Implementation
* The Effectiveness of Mindfulness-Based Interventions in Mental Health
* Exploring the Relationship Between Nutrition and Chronic Diseases
* The Future of Drug Development: Integrating AI in Clinical Trials
* Barriers to Accessing Mental Health Services in Low-Income Communities
* The Role of Pharmacogenomics in Personalized Medicine
* Emerging Infectious Diseases: Lessons Learned from COVID-19
* Patient-Centered Care: Strategies for Improving Healthcare Outcomes
* The Importance of Cultural Competence in Healthcare Delivery
* Telemedicine's Role in Managing Chronic Conditions
* Impact of Health Literacy on Patient Outcomes
* The Use of Big Data in Predicting Disease Outbreaks
* Innovative Approaches to Pain Management in Chronic Illness
* Ethical Implications of Genetic Editing in Human Embryos
* The Role of Community Health Workers in Improving Health Disparities
* Assessing the Impact of Vaccination Programs on Public Health
* Understanding Antimicrobial Resistance: Causes and Solutions
* The Influence of Social Determinants on Health Outcomes
* Integrative Approaches in Treating Mental Health Disorders
* The Effect of Physical Activity on Mental Health and Well-Being
* Barriers to Adherence in Chronic Disease Management
* The Role of Technology in Enhancing Patient Education
* Exploring the Relationship Between Sleep Disorders and Chronic Diseases
* Strategies for Managing Diabetes in Diverse Populations
* The Use of Mobile Health Apps for Chronic Disease Management
* Investigating the Link Between Stress and Immune Response
* Telehealth for Pediatric Care: Opportunities and Challenges
* The Role of Artificial Intelligence in Diagnosing Rare Diseases
* Evaluating the Impact of Mental Health Campaigns on Public Awareness
* The Importance of Interprofessional Collaboration in Healthcare
* Crisis Management in Healthcare: Lessons from the Pandemic
* The Role of Genetics in Cancer Treatment and Prevention
* Addressing the Opioid Crisis: Strategies for Prevention and Management
* The Evolving Role of Nurses in Patient Education and Advocacy
* The Impact of Social Media on Health Communication
* Examining the Efficacy of Teletherapy in Treating Anxiety Disorders
* Health Systems Strengthening: Strategies for Developing Countries
* Patient Safety: Analyzing Error Reporting and Prevention Strategies
* The Role of Immunotherapy in Cancer Treatment
* Challenges in End-of-Life Care: Palliative vs. Curative Approaches
* The Impact of Climate Change on Public Health
* Exploring the Role of Virtual Reality in Pain Management
* Assessing the Effectiveness of Health Promotion Interventions
* The Role of Artificial Intelligence in Mental Health Diagnostics
* Patient Privacy and Data Security in Telehealth Services
* Investigating the Role of Public Health Campaigns in Smoking Cessation



In [12]:
pdf_directory = "/content/drive/MyDrive/Research"
output_directory = "/content/drive/MyDrive/Medical_Data/Research_Papers"

os.makedirs(output_directory, exist_ok=True)

def convert_pdf_to_text(pdf_path, output_path):
    doc = fitz.open(pdf_path)
    text = ""

    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()

    with open(output_path, "w", encoding="utf-8") as text_file:
        text_file.write(text)

for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, pdf_file)
        output_path = os.path.join(output_directory, pdf_file.replace(".pdf", ".txt"))
        convert_pdf_to_text(pdf_path, output_path)
        print(f"Converted {pdf_file} to text and saved as {output_path}")

print("All PDFs have been converted to text files and saved.")


Converted Investigating the Link Between Stress and Immune Response.pdf to text and saved as /content/drive/MyDrive/Medical_Data/Research_Papers/Investigating the Link Between Stress and Immune Response.txt
Converted The Effectiveness of Mindfulness-Based Interventions in Mental Health .pdf to text and saved as /content/drive/MyDrive/Medical_Data/Research_Papers/The Effectiveness of Mindfulness-Based Interventions in Mental Health .txt
Converted Virtual_reality_and_pain_management_Current_trends.pdf to text and saved as /content/drive/MyDrive/Medical_Data/Research_Papers/Virtual_reality_and_pain_management_Current_trends.txt
Converted Investigating the Role of Public Health Campaigns in Smoking Cessation.pdf to text and saved as /content/drive/MyDrive/Medical_Data/Research_Papers/Investigating the Role of Public Health Campaigns in Smoking Cessation.txt
Converted Interprofessional_Team_Collaboration_in2.pdf to text and saved as /content/drive/MyDrive/Medical_Data/Research_Papers/Interp

In [13]:
#Combine the text files to one
def combine_txt_files(input_directory):
    output_directory = '//content/drive/MyDrive/Medical_Data/Research_Papers'
    output_file = os.path.join(output_directory, 'Combined_RP.txt')

    combined_content = []

    for filename in os.listdir(input_directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(input_directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                combined_content.append(file.read())

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write('\n\n'.join(combined_content))  #

    print(f"Combined file saved to: {output_file}")

input_directory = '/content/drive/MyDrive/Medical_Data/Research_Papers'
combine_txt_files(input_directory)


Combined file saved to: //content/drive/MyDrive/Medical_Data/Research_Papers/Combined_RP.txt


In [14]:
#Data Pre-Processing
def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Initialize stemmer
    stemmer = PorterStemmer()

    # Stem the tokens
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)

    return preprocessed_text

file_path = '/content/drive/MyDrive/Medical_Data/Research_Papers/Combined_RP.txt'
preprocessed_data = preprocess_text(file_path)

preprocessed_file_path = '/content/drive/MyDrive/Medical_Data/Research_Papers/Preprocessed_RP.txt'
with open(preprocessed_file_path, 'w', encoding='utf-8') as file:
    file.write(preprocessed_data)

print(f"Preprocessed data saved to: {preprocessed_file_path}")

Preprocessed data saved to: /content/drive/MyDrive/Medical_Data/Research_Papers/Preprocessed_RP.txt


WHO Guidelines

In [15]:
guideline_urls = [
    'https://www.cigna.com/knowledge-center/preventive-care',
    'https://www.who.int/health-topics/vaccines-and-immunization#tab=tab_1',
    'https://www.who.int/health-topics/vaccines-and-immunization#tab=tab_2',
    'https://www.who.int/health-topics/vaccines-and-immunization#tab=tab_3',
    'https://www.ucsfhealth.org/education/healthy-lifestyles-healthy-outlook',
    'https://www.cdc.gov/chronic-disease/living-with/index.html',
    'https://www.osha.gov/medical-first-aid/recognition',
    'https://www.osha.gov/medical-first-aid/programs',
    'https://www.who.int/news-room/fact-sheets/detail/cardiovascular-diseases-(cvds)',
    'https://www.who.int/southeastasia/activities/management-of-diabetes',
    'https://www.who.int/news-room/fact-sheets/detail/diabetes',
    'https://www.who.int/health-topics/chronic-respiratory-diseases#tab=tab_2',
    'https://www.guidelinecentral.com/guideline/309707/#section-311143',
    'https://www.idsociety.org/practice-guideline/covid-19-guideline-treatment-and-management/',
    'https://www.cancer.org/cancer/screening/american-cancer-society-guidelines-for-the-early-detection-of-cancer.html',
    'https://psychiatryonline.org/books/guidelines',
    'https://www.lonestar.edu/21249.htm',
    'https://www.mayoclinic.org/healthy-lifestyle/stress-management/in-depth/stress-relievers/art-20047257',
    'https://www.cdc.gov/nccdphp/dnpao/features/healthy-eating-tips/index.html',
    'https://www.stmaryshealthcaresystem.org/health-and-wellness/nutrition/special-diets',
    'https://www.fsis.usda.gov/food-safety/safe-food-handling-and-preparation/food-safety-basics/steps-keep-food-safe',
    'https://www.mayoclinic.org/healthy-lifestyle/pregnancy-week-by-week/in-depth/prenatal-care/art-20044882',
    'https://choc.org/primary-care/ages-stages/',
    'https://www.pennmedicine.org/for-patients-and-visitors/find-a-program-or-service/obstetrics/breastfeeding-services/care-guides/breastfeeding-basics#:~:text=Feeding%20Patterns&text=Breast%20milk%20is%20quickly%20digested,the%20first%20signs%20of%20hunger.',
    'https://www.nia.nih.gov/health/caregiving/services-older-adults-living-home',
    'https://www.nia.nih.gov/health/brain-health/cognitive-health-and-older-adults#:~:text=Cognitive%20health%20is%20the%20ability,factors%20contribute%20to%20cognitive%20health.'
]


In [16]:
#Webscrapping the websites
drug_directory = '/content/drive/MyDrive/Medical_Data/Guidelines'
os.makedirs(drug_directory, exist_ok=True)

def save_text_from_url(url, filename):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text(separator='\n')
        with open(os.path.join(drug_directory, filename), 'w', encoding='utf-8') as f:
            f.write(text)

        print(f"Saved: {filename}")

    except Exception as e:
        print(f"Error saving {filename}: {e}")

for i, url in enumerate(drug_urls):
    save_text_from_url(url, f'guidelines_info_{i}.txt')


Saved: guidelines_info_0.txt
Saved: guidelines_info_1.txt
Saved: guidelines_info_2.txt
Saved: guidelines_info_3.txt
Saved: guidelines_info_4.txt
Saved: guidelines_info_5.txt
Saved: guidelines_info_6.txt
Saved: guidelines_info_7.txt
Saved: guidelines_info_8.txt
Saved: guidelines_info_9.txt
Saved: guidelines_info_10.txt
Saved: guidelines_info_11.txt
Saved: guidelines_info_12.txt
Saved: guidelines_info_13.txt
Saved: guidelines_info_14.txt
Saved: guidelines_info_15.txt
Saved: guidelines_info_16.txt
Saved: guidelines_info_17.txt
Saved: guidelines_info_18.txt
Saved: guidelines_info_19.txt
Saved: guidelines_info_20.txt
Saved: guidelines_info_21.txt
Saved: guidelines_info_22.txt
Saved: guidelines_info_23.txt
Saved: guidelines_info_24.txt
Saved: guidelines_info_25.txt
Saved: guidelines_info_26.txt
Saved: guidelines_info_27.txt
Saved: guidelines_info_28.txt
Saved: guidelines_info_29.txt
Saved: guidelines_info_30.txt
Saved: guidelines_info_31.txt
Saved: guidelines_info_32.txt


In [17]:
#Combine the text files to one
def combine_txt_files(input_directory):
    output_directory = '/content/drive/MyDrive/Medical_Data/Guidelines'
    output_file = os.path.join(output_directory, 'Combined_Guidelines.txt')

    combined_content = []

    for filename in os.listdir(input_directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(input_directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                combined_content.append(file.read())

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write('\n\n'.join(combined_content))  #

    print(f"Combined file saved to: {output_file}")

input_directory = '/content/drive/MyDrive/Medical_Data/Guidelines'
combine_txt_files(input_directory)


Combined file saved to: /content/drive/MyDrive/Medical_Data/Guidelines/Combined_Guidelines.txt


In [18]:
#Data Pre-Processing
def preprocess_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Lowercase the text
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Initialize stemmer
    stemmer = PorterStemmer()

    # Stem the tokens
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)

    return preprocessed_text

file_path = '/content/drive/MyDrive/Medical_Data/Guidelines/Combined_Guidelines.txt'
preprocessed_data = preprocess_text(file_path)

# Save the preprocessed data to a new file
preprocessed_file_path = '/content/drive/MyDrive/Medical_Data/Guidelines/Preprocessed_Guidelines.txt'
with open(preprocessed_file_path, 'w', encoding='utf-8') as file:
    file.write(preprocessed_data)

print(f"Preprocessed data saved to: {preprocessed_file_path}")

Preprocessed data saved to: /content/drive/MyDrive/Medical_Data/Guidelines/Preprocessed_Guidelines.txt


***INDEXING***

In [19]:
#Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
#Load documents from all specified directories
def load_documents_from_multiple_folders(base_directory, sub_folders):
    document_contents = []
    document_filenames = []
    for sub_folder in sub_folders:
        folder_path = os.path.join(base_directory, sub_folder)
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                    document_contents.append(file.read())
                    document_filenames.append(filename)
    return document_contents, document_filenames

In [21]:
#Generate embeddings for your documents
def generate_embeddings(documents):
    return model.encode(documents, convert_to_tensor=True)

In [22]:
#Create the FAISS index
def create_faiss_index(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings.cpu().numpy())
    return index

In [23]:
base_directory = '/content/drive/My Drive/Medical_Data/'
sub_folders = ['Disease', 'Drug', 'Research_Papers', 'Guidelines']

In [24]:
document_contents, document_filenames = load_documents_from_multiple_folders(base_directory, sub_folders)
embeddings = generate_embeddings(document_contents)

In [25]:
faiss_index = create_faiss_index(embeddings)

In [26]:
faiss_index_file_path = '/content/drive/My Drive/Medical_Data/faiss_index.index'
faiss.write_index(faiss_index, faiss_index_file_path)
print(f"FAISS index created and saved to: {faiss_index_file_path}")


FAISS index created and saved to: /content/drive/My Drive/Medical_Data/faiss_index.index


In [27]:
index_mapping_file_path = '/content/drive/My Drive/Medical_Data/document_filenames.json'
with open(index_mapping_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(document_filenames, json_file, ensure_ascii=False, indent=4)
print(f"Document filenames saved to: {index_mapping_file_path}")


Document filenames saved to: /content/drive/My Drive/Medical_Data/document_filenames.json
